# Homework 2 #
## Word count ##
### Stupid version ###

In [1]:
from pyspark import SparkContext, SparkConf

# Spark Setup
conf = SparkConf().setAppName('Word Count').setMaster('local')
sc = SparkContext(conf=conf)

In [2]:
docs = sc.textFile('dataset.txt', 12).cache()  #returns an RDD (there are at least 12 partitions), and saves it to cache
print("Number of documents: ", docs.count())
docs.collect()

Number of documents:  5


['prova a dire cosa prova a di dire su io su',
 'capra capra capra mia bella bella ti vagone vagone tu ta',
 'lupo cattivo oltre il buio buio albero tuta tu ta ti',
 'marta odia marta che odia marta ciao bubu bubu io ti',
 'sopra la panca la capra canta sotto la panca la capra crepa']

In [3]:
words = docs.flatMap(lambda doc: doc.split())
print("Number of words: ", words.count())
words.collect()

Number of words:  56


['prova',
 'a',
 'dire',
 'cosa',
 'prova',
 'a',
 'di',
 'dire',
 'su',
 'io',
 'su',
 'capra',
 'capra',
 'capra',
 'mia',
 'bella',
 'bella',
 'ti',
 'vagone',
 'vagone',
 'tu',
 'ta',
 'lupo',
 'cattivo',
 'oltre',
 'il',
 'buio',
 'buio',
 'albero',
 'tuta',
 'tu',
 'ta',
 'ti',
 'marta',
 'odia',
 'marta',
 'che',
 'odia',
 'marta',
 'ciao',
 'bubu',
 'bubu',
 'io',
 'ti',
 'sopra',
 'la',
 'panca',
 'la',
 'capra',
 'canta',
 'sotto',
 'la',
 'panca',
 'la',
 'capra',
 'crepa']